In [1]:
import os
import pickle
import torch
import numpy as np
# from transformers import *

In [22]:
# load data
data_dir = '/home-3/msomme16@jhu.edu/scratch/shimmer/data/'
CDS_path = os.path.join(data_dir, "CDS_3600.pkl")

with open(CDS_path, 'rb') as f:
    CDS = pickle.load(f)
print(CDS[10])

MKIAFDTVNVYYLPQFIPICDELAKRGHEVKLVCYSNKNKAQAFEQVLLSFGYEFCWVDDDKAARDLYLKEEPDWIFFGNGFSYLDDIHKVSKTAQLGHGIGPKPSYYHKSSTPMTVRFIEGKMRLAKIRELYPNDEFVQVGFSKLDPLFNNTEPGLKYDELGLDKGKPTLLFAPTFNPSSLECFPDDWPSHFTDFNILIKPHTFTYSREAYKNQRKKLKKWAQFNNTYVATETDISLLPFMKDADILISEASSTLFEFVALSKPVIVCNFFKLKWSYRGIFNYRFEKRFGKDNVIYKNIGLHINDYTELRAAIDKQLNNEHLYKEERKNYTQDHVGPTDGKSSLRIVDYIENN


In [ ]:
# import logging
# logging.basicConfig(level=logging.INFO)

In [7]:
from transformers import BertConfig, BertModel, BertForMaskedLM

# custom parameters for BERT model
vocab_size = 30522 # 10 # Vocabulary size of inputs_ids in BertModel. default=30522
hidden_size = 768 # Size of the encoder layers and the pooler layer, default=768
num_hidden_layers = 12 # Number of hidden layers in the Transformer encoder. default=12
num_attention_heads = 12 # Number of attention heads for each attention layer in the Transformer encoder, default=12
intermediate_size = 3072 # The size of the “intermediate” (i.e., feed-forward) layer in the Transformer encoder. default=3072
hidden_act = "gelu" # The non-linear activation function (function or string) in the encoder and pooler. If string, “gelu”, “relu”, “swish” and “gelu_new” are supported. default="gelu"
hidden_dropout_prob = 0.1 # The dropout probabilitiy for all fully connected layers in the embeddings, encoder, and pooler. default=0.1
attention_probs_dropout_prob = 0.1 # The dropout ratio for the attention probabilities. default=0.1
max_position_embeddings = 512 # 5000 # The maximum sequence length that this model might ever be used with. Typically set this to something large just in case (e.g., 512 or 1024 or 2048). default=512
type_vocab_size = 2 # 1 # The vocabulary size of the token_type_ids passed into BertModel. default=2
initializer_range = 0.02 # The sttdev of the truncated_normal_initializer for initializing all weight matrices. default=0.02
layer_norm_eps = 1e-12 # The epsilon used by LayerNorm. default=1e-12


config = BertConfig(vocab_size_or_config_json_file=vocab_size,
                    hidden_size=hidden_size,
                    num_hidden_layers=num_hidden_layers,
                    num_attention_heads=num_attention_heads,
                    intermediate_size=intermediate_size,
                    hidden_act=hidden_act,
                    hidden_dropout_prob=hidden_dropout_prob,
                    attention_probs_dropout_prob=attention_probs_dropout_prob,
                    max_position_embeddings=max_position_embeddings,
                    type_vocab_size=type_vocab_size,
                    initializer_range=initializer_range,
                    layer_norm_eps=layer_norm_eps)

# model = BertModel(config)
model = BertForMaskedLM(config)

print(model)
model.to('cuda')


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [24]:
# encode data as GPU tensors
max_aa_seq_length = max_position_embeddings


def tokenize_aa_seq_murphy10(aa_seq):
    table = {"L":1,
             "V":1,
             "I":1,
             "M":1,
             "C":2,
             "A":3,
             "G":4,
             "S":5,
             "T":5,
             "P":6,
             "F":7,
             "Y":7,
             "W":7,
             "E":8,
             "D":8,
             "N":8,
             "Q":8,
             "K":9,
             "R":9,
             "H":10}
    tokenized = [table[aa] for aa in aa_seq]
    return tokenized
    
example_small = CDS[0:2]

tokens = [tokenize_aa_seq_murphy10(seq) for seq in example_small]
tokens_tensor = torch.zeros(len(tokens), max_aa_seq_length, dtype=torch.long)
for i in range(len(tokens)):
    for j in range(len(tokens[i])):
#     for j in range(10):
        tokens_tensor[i][j] += tokens[i][j]

tokens_tensor = tokens_tensor.to('cuda')

print(tokens_tensor)
print(tokens_tensor.shape)

tensor([[1, 9, 8,  ..., 0, 0, 0],
        [1, 8, 4,  ..., 0, 0, 0]], device='cuda:0')
torch.Size([2, 512])


In [ ]:
# train model
import torch.optim as optim
import torch.nn.functional as F

def train_epoch(epoch, args, model, device, data_loader, optimizer):
    model.train()  # set to training mode, disappointingly does not actually train the model 
    pid = os.getpid()
    for batch_idx, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = F.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('{}\tTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                pid, epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()))
            
optimizer = optim.SGD(model.parameters(), lr=0.01)#, momentum=args.momentum)
optimizer.zero_grad()
# output = model(tokens_tensor)
# loss = F.nll_loss(output, target.to('cuda'))
print([x for x in model.parameters()])


In [14]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model = BertForMaskedLM(config)

input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]
print(loss)
print(prediction_scores)

tensor(10.3736, grad_fn=<NllLossBackward>)
tensor([[[-0.1017, -0.4088, -0.0420,  ..., -0.5752, -0.2629,  0.0583],
         [ 0.5595, -0.9000,  0.0395,  ..., -0.9279, -0.3495,  0.1374],
         [-0.2054, -0.1488, -0.8590,  ..., -0.5412,  1.6125,  0.9725],
         [ 0.8595,  0.0048,  0.1969,  ..., -0.1552,  0.2289,  0.0870],
         [-0.1352, -0.0248,  0.4349,  ...,  0.2199, -0.2500, -0.0651],
         [ 0.0803,  0.1161, -0.5024,  ..., -0.2887,  0.8975,  0.4796]]],
       grad_fn=<AddBackward0>)


In [36]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model = BertForMaskedLM(config)
model.to('cuda')

# input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
# input_ids = tokens_tensor[0].unsqueeze(0)
input_ids = tokens_tensor[0:2]

outputs = model(input_ids, masked_lm_labels=input_ids)
loss, prediction_scores = outputs[:2]
print(loss)
print(prediction_scores.shape)

tensor(10.3518, device='cuda:0', grad_fn=<NllLossBackward>)
torch.Size([2, 512, 30522])


In [33]:
print(input_ids.shape)
print(tokens_tensor[0].unsqueeze(0).shape)
print(tokens_tensor[0:2].shape)

torch.Size([1, 512])
torch.Size([1, 512])
torch.Size([2, 512])


In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
# example tokenization
from transformers import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

masked_index = 8
tokenized_text[masked_index] = '[MASK]'
print(tokenized_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)
print(tokens_tensor.shape)

In [ ]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
# segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
#     outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

In [ ]:
# GPT-2

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')

# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2-large')


# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()
model.to('cuda')

# original_text = "Martin Steinegger is in Peru because he "
# original_text = "The real reason Steven keeps recruiting German postdocs is "
original_text = "The secret to giving a fun and compelling Joint Lab Meeting presentation is "
# original_text = "Johns Hopkins University is"
# original_text = "UC Berkeley is"
# original_text = "Why did Donald Trump "
# original_text = "Finding genes is easy... The secret is "
# original_text = "The reason I want to get a PhD in Biomedical Engineering is "
# original_text = "Computational gene finding is easy, the real secret is "
# original_text = "Computational gene finding is easy, the problem is "
# original_text = "I came to Johns Hopkins University because "
# original_text = "My experiment does not work because "





text = original_text
for i in range(100): # not the best way to iterate, but it works
#     if text[-1] != ".":
        # Encode a text inputs
        indexed_tokens = tokenizer.encode(text)

        # Convert indexed tokens in a PyTorch tensor
        tokens_tensor = torch.tensor([indexed_tokens])

        # If you have a GPU, put everything on cuda
        tokens_tensor = tokens_tensor.to('cuda')

        # Predict all tokens
        with torch.no_grad():
            outputs = model(tokens_tensor)
            predictions = outputs[0]

        # get the predicted next sub-word (in our case, the word 'man')
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

        text = predicted_text

In [ ]:
print("Original text:\t\t", original_text)
print("Completed sentence:\t", predicted_text)